In [ ]:
# %help

UsageError: Line magic function `%status` not found.


In [ ]:
%%configure -f
{
    "JOB_NAME": "jamestest_notebook",
    "TempDir": "s3://aws-glue-assets-734176943427-cn-northwest-1/temporary/"
}

: 

In [ ]:
# include extra lib
# %extra_jars s3://aws-glue-assets-734176943427-cn-northwest-1/dependent_jars/spark-sql-kinesis_2.12-1.2.0_spark-3.0.jar,s3://aws-glue-assets-734176943427-cn-northwest-1/dependent_jars/spark-hadoop-cloud_2.13-3.2.1.jar,s3://aws-glue-assets-734176943427-cn-northwest-1/dependent_jars/aws-java-sdk-1.11.848.jar
%streaming
%number_of_workers 2
%worker_type G.1X

In [ ]:
import sys
import pandas as pd
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job
import logging
from pyspark.sql import DataFrame, Row
from pyspark.sql.functions import from_json
import datetime
from awsglue import DynamicFrame
import json



args = getResolvedOptions(sys.argv, ["JOB_NAME"])
# sc = SparkContext()
sc = SparkContext.getOrCreate();

glueContext = GlueContext(sc)
spark = glueContext.spark_session
print("Load Session OK")

In [ ]:
from pyspark.sql.types import *

recipientSchema = StructType([
    StructField("firstName",StringType(),True), 
    StructField("lastName",StringType(),True), 
    StructField("phoneNumber", StringType(), True)
])

addressSchema = StructType([
    StructField("address1",StringType(),True), 
    StructField("city",StringType(),True), 
    StructField("state", StringType(), True),
    StructField("postalCode", StringType(), True),
    StructField("country", StringType(), True),
    StructField("county", StringType(), True)
])

getBySchema = StructType([
    StructField("maxDate", StructType([
                            StructField("dateTime",StringType(),True), 
                            StructField("timezone",StringType(),True), 
                            StructField("precision", StringType(), True)
                        ]),
                True)
])

shippingSchema = StructType([ 
    StructField("recipient", recipientSchema, True), 
    StructField("address", addressSchema, True), 
    StructField("method", StringType(), True), 
    StructField("getBy", getBySchema, True)
  ])

validationSummarySchema = StructType([ 
    StructField("result", StringType(), True), 
    ])

totalsSchema = StructType([
    StructField("items", 
                StructType([ 
                    StructField("total", DecimalType(), True), 
                    StructField("details", 
                                StructType([ 
                                    StructField("price", DecimalType(), True), 
                                    StructField("discount", DecimalType(), True)
                                ]), True), 
                    ]),
                True), 
])

entrySchema = StructType() \
 .add("id", StructType([StructField("S", StringType())])) \
 .add("appId", StructType([StructField("S", StringType())]), True) \
 .add("channel", StructType([StructField("S", StringType())]), True) \
 .add("checkoutId", StructType([StructField("S", StringType())]), True) \
 .add("creationTimestamp", StructType([StructField("N", StringType())]), True) \
 .add("currency", StructType([StructField("S", StringType())]), True) \
 .add("forwardedFor", StructType([StructField("S", StringType())]), True) \
 .add("geolocation", StructType([StructField("S", StringType())]), True) \
 .add("launchId", StructType([StructField("S", StringType())]), True) \
 .add("launchSkuStatus", StructType([StructField("S", StringType())])) \
 .add("launchSkuStatusValidation", StructType([StructField("S", StringType())]), True) \
 .add("paymentStatus", StructType([StructField("S", StringType())]), True) \
 .add("paymentToken", StructType([StructField("S", StringType())]), True) \
 .add("postpayLink", StructType([StructField("S", StringType())]), True) \
 .add("postpayRetry", StructType([StructField("BOOL", BooleanType())]), True) \
 .add("locale", StructType([StructField("S", StringType())]), True) \
 .add("rank", StructType([StructField("N", StringType())]), True) \
 .add("reentryPermitted", StructType([StructField("BOOL", BooleanType())]), True) \
 .add("reservationId", StructType([StructField("S", StringType())]), True) \
 .add("reservationJobId", StructType([StructField("S", StringType())]), True) \
 .add("reservationTimestamp", StructType([StructField("N", StringType())]), True) \
 .add("retailPickupPerson", StructType([StructField("S", StringType())]), True) \
 .add("selectedTimestamp", StructType([StructField("N", StringType())]), True) \
 .add("shipping", StructType([StructField("S", StringType())]), True) \
 .add("skuId", StructType([StructField("S", StringType())]), True) \
 .add("sort", StructType([StructField("N", StringType())]), True) \
 .add("status", StructType([StructField("S", StringType())]), True) \
 .add("totals", StructType([StructField("S", StringType())]), True) \
 .add("traceContext", StructType([StructField("S", StringType())]), True) \
 .add("trueClientIp", StructType([StructField("S", StringType())]), True) \
 .add("ttl", StructType([StructField("N", StringType())]), True) \
 .add("upmId", StructType([StructField("S", StringType())]), True) \
 .add("userAgent", StructType([StructField("S", StringType())]), True) \
 .add("userType", StructType([StructField("S", StringType())]), True) \
 .add("validationSummary", StructType([StructField("S", StringType())]), True) \
 .add("waitingReason", StructType([StructField("S", StringType())]), True) 
 

ddbSchema = StructType() \
    .add("ApproximateCreationDateTime", StringType()) \
    .add("Keys", StructType([StructField("id", StructType([StructField("S", StringType())]))])) \
    .add("NewImage", entrySchema, True) \
    .add("OldImage", entrySchema, True) \
    .add("SizeBytes", IntegerType())


schema = StructType() \
 .add("awsRegion", StringType()) \
 .add("eventID", StringType()) \
 .add("eventName", StringType()) \
 .add("userIdentity", StringType()) \
 .add("recordFormat", StringType()) \
 .add("tableName", StringType()) \
 .add("dynamodb", ddbSchema) \
 .add("eventSource", StringType())

print("Load Schema OK")

In [ ]:
from pyspark.sql.functions import col,from_json, lpad, year, month, dayofmonth
from pyspark.sql import SparkSession,Row
# spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

#read json from text file
# dfFromTxt=spark.read.text("s3://bots-taijitu-intermediate-data-export/example.json.txt")
# dfFromTxt.show(truncate=False)

kinesisDF = spark \
  .readStream \
  .format("kinesis") \
  .option("streamName", "launch-entry-eda-data-stream") \
  .option("endpointUrl", "https://kinesis.cn-northwest-1.amazonaws.com.cn") \
  .option("initialPosition", "earliest") \
  .option("region", "cn-northwest-1") \
  .option("inferschema",True) \
  .load()

# logger.info('Hello')
print('Load Kinesis OK')
# kinesisDF.printSchema()

selectedColumns = ["id", "appId", "channel", "checkoutId", "creationTimestamp", "currency", "forwardedFor", "geolocation", "launchId", "launchSkuStatus", 
                   "launchSkuStatusValidation", "locale", "paymentStatus", "paymentToken", "postpayLink", "postpayRetry", "rank", "reentryPermitted", 
                   "reservationId", "reservationJobId", "reservationTimestamp", "retailPickupPerson", "selectedTimestamp", "skuId", "sort", "status", 
                   "upmId", "ttl", "traceContext", "trueClientIp", "userAgent", "userType", "waitingReason", "validationSummary_result", 
                   "shipping_recipient_firstName", "shipping_recipient_lasttName", "shipping_recipient_phoneNumber",
                   "shipping_address_address1", "shipping_address_city", "shipping_address_state", "shipping_address_postalCode", "shipping_address_country", "shipping_address_county",
                   "shipping_method",
                   "year", "month", "day",
                   "metadata_eventId", "metadata_eventName", "metadata_tableName", 
                   "db_item_creationDateTime", "db_item_key"
                  ]

checkpoint_location="s3://aws-glue-assets-734176943427-cn-northwest-1/temporary/jamestest_notebook/checkpoint/"
output_location="s3://bots-taijitu-intermediate-data-export/launch-entries/"
from pyspark.sql.functions import col,from_json
dfJSON = kinesisDF \
    .selectExpr("CAST(data AS STRING)") \
    .withColumn("jsonData",from_json(col("data"),schema)) \
    .withColumn("sh", from_json(col("jsonData.dynamodb.NewImage.shipping.S"), shippingSchema))\
    .withColumn("valSummary", from_json(col("jsonData.dynamodb.NewImage.validationSummary.S"), validationSummarySchema)) \
    .withColumn("metadata_eventId", col("jsonData.eventID"))\
    .withColumn("metadata_eventName", col("jsonData.eventName"))\
    .withColumn("metadata_tableName", col("jsonData.tableName"))\
    .withColumn("db_item_creationDateTime", col("jsonData.dynamodb.ApproximateCreationDateTime"))\
    .withColumn("db_item_key", col("jsonData.dynamodb.Keys.id.S"))\
    .withColumn("id", col("jsonData.dynamodb.NewImage.id.S"))\
    .withColumn("appId", col("jsonData.dynamodb.NewImage.appId.S"))\
    .withColumn("channel", col("jsonData.dynamodb.NewImage.channel.S"))\
    .withColumn("checkoutId", col("jsonData.dynamodb.NewImage.checkoutId.S"))\
    .withColumn("creationTimestamp", col("jsonData.dynamodb.NewImage.creationTimestamp.N"))\
    .withColumn("currency", col("jsonData.dynamodb.NewImage.currency.S"))\
    .withColumn("forwardedFor", col("jsonData.dynamodb.NewImage.forwardedFor.S"))\
    .withColumn("geolocation", col("jsonData.dynamodb.NewImage.geolocation.S"))\
    .withColumn("launchId", col("jsonData.dynamodb.NewImage.launchId.S"))\
    .withColumn("launchSkuStatus", col("jsonData.dynamodb.NewImage.launchSkuStatus.S"))\
    .withColumn("launchSkuStatusValidation", col("jsonData.dynamodb.NewImage.launchSkuStatusValidation.S"))\
    .withColumn("locale", col("jsonData.dynamodb.NewImage.locale.S"))\
    .withColumn("paymentStatus", col("jsonData.dynamodb.NewImage.paymentStatus.S"))\
    .withColumn("paymentToken", col("jsonData.dynamodb.NewImage.paymentToken.S"))\
    .withColumn("postpayLink", col("jsonData.dynamodb.NewImage.postpayLink.S"))\
    .withColumn("postpayRetry", col("jsonData.dynamodb.NewImage.postpayRetry.BOOL"))\
    .withColumn("rank", col("jsonData.dynamodb.NewImage.rank.N"))\
    .withColumn("reentryPermitted", col("jsonData.dynamodb.NewImage.reentryPermitted.BOOL"))\
    .withColumn("reservationId", col("jsonData.dynamodb.NewImage.reservationId.S"))\
    .withColumn("reservationJobId", col("jsonData.dynamodb.NewImage.reservationJobId.S"))\
    .withColumn("reservationTimestamp", col("jsonData.dynamodb.NewImage.reservationTimestamp.N"))\
    .withColumn("retailPickupPerson", col("jsonData.dynamodb.NewImage.retailPickupPerson.S"))\
    .withColumn("selectedTimestamp", col("jsonData.dynamodb.NewImage.selectedTimestamp.N"))\
    .withColumn("skuId", col("jsonData.dynamodb.NewImage.skuId.S"))\
    .withColumn("sort", col("jsonData.dynamodb.NewImage.sort.N"))\
    .withColumn("status", col("jsonData.dynamodb.NewImage.status.S"))\
    .withColumn("traceContext", col("jsonData.dynamodb.NewImage.traceContext.S"))\
    .withColumn("trueClientIp", col("jsonData.dynamodb.NewImage.trueClientIp.S"))\
    .withColumn("ttl", col("jsonData.dynamodb.NewImage.ttl.N"))\
    .withColumn("upmId", col("jsonData.dynamodb.NewImage.upmId.S"))\
    .withColumn("userAgent", col("jsonData.dynamodb.NewImage.userAgent.S"))\
    .withColumn("validationSummary_result", col("jsonData.dynamodb.NewImage.validationSummary.S"))\
    .withColumn("waitingReason", col("jsonData.dynamodb.NewImage.waitingReason.S"))\
    .withColumn("userType", col("jsonData.dynamodb.NewImage.userType.S"))\
    .withColumn("validationSummary_result", col("valSummary.result"))\
    .withColumn("shipping_recipient_firstName", col("sh.recipient.firstName"))\
    .withColumn("shipping_recipient_lasttName", col("sh.recipient.lastName"))\
    .withColumn("shipping_recipient_phoneNumber", col("sh.recipient.phoneNumber"))\
    .withColumn("shipping_address_address1", col("sh.address.address1"))\
    .withColumn("shipping_address_city", col("sh.address.city"))\
    .withColumn("shipping_address_state", col("sh.address.state"))\
    .withColumn("shipping_address_postalCode", col("sh.address.postalCode"))\
    .withColumn("shipping_address_country", col("sh.address.country"))\
    .withColumn("shipping_address_county", col("sh.address.county"))\
    .withColumn("shipping_method", col("sh.method"))\
    .withColumn("year", year((col("jsonData.dynamodb.NewImage.creationTimestamp.N")/ 1000).cast(TimestampType())))\
    .withColumn("month", lpad(month((col("jsonData.dynamodb.NewImage.creationTimestamp.N")/ 1000).cast(TimestampType())), 2, "0"))\
    .withColumn("day", lpad(dayofmonth((col("jsonData.dynamodb.NewImage.creationTimestamp.N")/ 1000).cast(TimestampType())), 2, "0"))\
    .select(selectedColumns)\
    .drop("jsonData.dynamodb.NewImage.shipping", "jsonData.dynamodb.NewImage.totals", "jsonData.dynamodb.NewImage.validationSummary") \
    .writeStream \
    .outputMode('append') \
    .format("csv") \
    .partitionBy("year", "month", "day") \
    .option("header", True) \
    .option("checkpointLocation", checkpoint_location) \
    .option("path", output_location) \
    .trigger(processingTime="30 seconds") \
    .start() \
    .awaitTermination()


# Printt original data
# dfJSON = kinesisDF\
#     .selectExpr('CAST(data AS STRING)')\
#     .select(from_json('data', schema).alias('data'))\
#     .select('data.*')\
#     .writeStream\
#     .outputMode('append')\
#     .format('console')\
#     .option("checkpointLocation", checkpoint_location)\
#     .trigger(once=True) \
#     .start()\
#     .awaitTermination()

print('Finished')